In [1]:
import gym
import numpy as np
import random as rd
import time
from gym.envs.registration import register
from IPython.display import clear_output


In [2]:
try:
    register(
        id="FrozenLakeNoSlip-v0",
        entry_point="gym.envs.toy_text:FrozenLakeEnv",
        kwargs={"map_name": "4x4", 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.70,  # optimum = 0.74
    )
except:
    pass


#import environment
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print(f"Obsarvation_space: {env.observation_space}")
print(f"action space low limit: {env.action_space}")
print(f"action space high limit: {env.action_space}")

type(env.action_space)


Obsarvation_space: Discrete(16)
action space low limit: Discrete(4)
action space high limit: Discrete(4)


gym.spaces.discrete.Discrete

In [3]:
class Agent():
    def __init__(self, env):
        #Initialisation
        self.env = env
        # check if the value is descrete or continous
        print(type(env.action_space))
        self.discrete =\
            type(env.action_space) == gym.spaces.discrete.Discrete
        print(type(env.action_space))
        print(self.discrete)
        
        if (self.discrete): # discrete
            self.action_space_size = env.action_space.n
            print("Discrete")
        else:
            self.action_space_low = env.action_space.low # you add n at the end to give you the range
            self.action_space_high = env.action_space.high
            self.shape = env.action_space.shape
        
    def get_action(self,state):
        if self.discrete:
            return rd.choice(range(self.action_space_size))
        else:
            action = np.random.uniform(low =self.action_space_low, high = self.action_space_high, size = self.shape)
            return(action)


In [59]:
class QAgent(Agent):
    def __init__(self,env,discount_rate = 0.97, learning_rate = 0.01):
        super().__init__(env)
        print("QAgent")
        self.action_space_size = env.action_space.n
        self.state_size = env.observation_space.n
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.opsilon = 1.0
        
        # build a demi model to visualize and train the model
        self.build_model()

    def build_model(self):
        print("build_model")

        self.q_table = 1e-4*np.random.random([self.state_size, self.action_space_size]) # This is very important for the initilization
        print(self.q_table)
        print(self.q_table.shape)

    def get_action(self,state):
        print("Get Action")
        # calculate the expected value
        print(f"state: {state}")
        q_curr_state = self.q_table[state]
        probability = np.random.uniform(0,1)

        if probability >= self.opsilon:
            action = np.argmax(q_curr_state)
        else:
            action = super().get_action(state)
        # the best action would be the max if its sourounding ==> we get the index if the loc 
        # see that i
        return(action)
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_curr_state = self.q_table[state]
        #Next state 
     
        q_next_state = self.q_table[next_state]
        q_next_state = np.zeros(self.action_space_size) if done else q_next_state
        
        #target
        q_target = reward + self.discount_rate* np.max(q_next_state)

        #update : calculate the distance from the current state, to the target
        q_update = q_target - self.q_table[state, action]
        print(f"q_update: {q_update}")
        self.q_table[state,action] += self.learning_rate *(q_update)

        if done:
            self.opsilon  =  self.opsilon *0.99
        

agent = QAgent(env)

<class 'gym.spaces.discrete.Discrete'>
<class 'gym.spaces.discrete.Discrete'>
True
Discrete
QAgent
build_model
[[3.78540349e-05 6.45617047e-05 9.29082145e-05 7.07756280e-05]
 [6.52924492e-05 8.97977905e-05 3.50011393e-06 3.05581661e-06]
 [6.06448341e-05 6.39205748e-05 8.95941315e-05 4.12417284e-06]
 [9.50085476e-05 9.35193343e-05 5.05308920e-05 4.35834113e-05]
 [3.69503930e-06 5.94743069e-05 8.62182728e-05 4.75508418e-05]
 [5.94504001e-05 5.34967531e-05 2.26687433e-05 9.68791931e-05]
 [6.68275940e-05 9.44861799e-05 4.70711762e-05 1.12707572e-05]
 [8.49293466e-05 7.41098209e-05 6.47117669e-05 3.44250480e-05]
 [6.72043912e-06 7.56433157e-05 2.55300837e-05 1.44516711e-05]
 [1.00083225e-05 7.08419322e-05 3.50406285e-05 7.41504453e-05]
 [3.57578901e-05 3.24591838e-05 3.19303281e-05 9.37783963e-05]
 [3.87451323e-06 9.18402928e-05 1.28545873e-05 7.02115027e-05]
 [8.66412053e-05 3.42083182e-05 5.25716410e-05 1.68757842e-05]
 [1.26540989e-06 5.40510569e-05 8.24960456e-05 2.77921328e-05]
 [3.651

In [62]:

agent = QAgent(env)

print(agent.get_action)
n_itters = 1000
total_reward = 0

for itter in range(n_itters):
    state =env.reset() #reset environment
    done = False
    while not done:
        #action
        curr_act = agent.get_action(state)
        print(f"Current_Action index: {curr_act}")
        #step : env.step(curr_act) returns the next step 
        state_next,reward, done, info = env.step(curr_act)

        experience_ = (state,curr_act,state_next,reward, done)
        print(f"Experience: {experience_}")
        # train 
        agent.train(experience = experience_)
        print(state)
        state = state_next
        total_reward += reward

        #state, action, next_state, reward, done
        print(f"state: {state}, action: {curr_act}, epsolin = {agent.opsilon}, total_reward: {total_reward}")
        #reward
        #print(agent.q_table)
        env.render()
        #score

        time.sleep(0.05)
        clear_output(wait=True) # flush the notbook outpur so I see the same thing over itself 

env.close()

Get Action
state: 0
Current_Action index: 0
Experience: (0, 0, 0, 0.0, False)
q_update: -2.2370954998849224e-06
0
state: 0, action: 0, epsolin = 0.17398982847626407, total_reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
